In [10]:
# To be able to make edits to repo without having to restart notebook
%load_ext autoreload
%autoreload 2
import sys 
sys.path.append('..')
from eegdash.signalstore_data_utils import SignalstoreBIDS
from eegdash.signalstore_data_utils import SignalstoreOpenneuro
import s3fs
import os
import tempfile
import mne
import pynwb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Test download from Openneuro S3

In [2]:
eegdash = SignalstoreOpenneuro(
    is_public=True,
    local_filesystem=False,
)
eegdash.get({'dataset': 'ds002718', 'subject': '002'})

Pinged your deployment. You successfully connected to MongoDB!
Found 1 records


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 10 concurrent workers.
/Users/dtyoung/Documents/EEGLAB/EEG-Dash-Data/tests/../eegdash/signalstore_data_utils.py:154: RuntimeWarning: Unknown types found, setting as type EEG:
ekg: ['EEG063', 'EEG064']
heog: ['EEG061']
veog: ['EEG062']
  EEG = mne.io.read_raw_eeglab(bids_file)
/Users/dtyoung/Documents/EEGLAB/EEG-Dash-Data/tests/../eegdash/signalstore_data_utils.py:154: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  EEG = mne.io.read_raw_eeglab(bids_file)
/Users/dtyoung/Documents/EEGLAB/EEG-Dash-Data/tests/../eegdash/signalstore_data_utils.py:154: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  EEG = mne.io.read_raw_eeglab(bids_file)


[<xarray.DataArray (channel: 74, time: 747750)> Size: 443MB
 array([[3.90708122e-05, 4.42323761e-05, 3.64698410e-05, ...,
         4.36852455e-05, 4.51106300e-05, 4.22494888e-05],
        [4.68087311e-05, 4.57453194e-05, 4.40404015e-05, ...,
         2.10300961e-05, 2.51476688e-05, 2.52572155e-05],
        [3.52780228e-05, 3.70137100e-05, 4.12147102e-05, ...,
         5.17162743e-05, 5.74016876e-05, 5.65963058e-05],
        ...,
        [7.02251358e-05, 7.77657928e-05, 6.62534866e-05, ...,
         6.97815170e-05, 1.01796478e-04, 9.74829407e-05],
        [6.03754272e-05, 2.69368916e-05, 3.46484642e-05, ...,
         6.66851044e-05, 6.78498688e-05, 6.35415993e-05],
        [3.64559059e-05, 4.87539520e-05, 4.18273659e-05, ...,
         4.89408951e-05, 6.06918449e-05, 5.85218697e-05]],
       shape=(74, 747750))
 Coordinates:
   * time     (time) float64 6MB 0.0 0.004 0.008 ... 2.991e+03 2.991e+03
   * channel  (channel) <U6 2kB 'EEG001' 'EEG002' 'EEG003' ... 'EEG073' 'EEG074']

Test download from SCCN S3 with Zarr

In [3]:
%%time
eegdash = SignalstoreBIDS(
    is_public=True,
    local_filesystem=False,
)
eegdash.get({'dataset': 'ds002718', 'subject': '002'})

Pinged your deployment. You successfully connected to MongoDB!
Found 1 records
CPU times: user 195 ms, sys: 46 ms, total: 241 ms
Wall time: 6.01 s


[<xarray.DataArray 'eeg_signal__ds002718_sub-002_task-FaceRecognition_eeg.set' (
                                                                                channel: 74,
                                                                                time: 747750)> Size: 443MB
 [55333500 values with dtype=float64]
 Coordinates:
   * channel  (channel) object 592B 'EEG001' 'EEG002' ... 'EEG073' 'EEG074'
   * time     (time) float64 6MB 0.0 0.004 0.008 ... 2.991e+03 2.991e+03
 Attributes:
     data_name:           ds002718_sub-002_task-FaceRecognition_eeg.set
     has_file:            True
     modality:            EEG
     run:                 
     sampling_frequency:  250
     schema_ref:          eeg_signal
     session:             
     subject:             2
     task:                FaceRecognition
     version_timestamp:   0]

In [4]:
data = eegdash.get({'dataset': 'ds002718', 'subject': '002'})

Found 1 records


In [5]:
data[0].to_numpy()

array([[3.90708122e-05, 4.42323761e-05, 3.64698410e-05, ...,
        4.36852455e-05, 4.51106300e-05, 4.22494888e-05],
       [4.68087311e-05, 4.57453194e-05, 4.40404015e-05, ...,
        2.10300961e-05, 2.51476688e-05, 2.52572155e-05],
       [3.52780228e-05, 3.70137100e-05, 4.12147102e-05, ...,
        5.17162743e-05, 5.74016876e-05, 5.65963058e-05],
       ...,
       [7.02251358e-05, 7.77657928e-05, 6.62534866e-05, ...,
        6.97815170e-05, 1.01796478e-04, 9.74829407e-05],
       [6.03754272e-05, 2.69368916e-05, 3.46484642e-05, ...,
        6.66851044e-05, 6.78498688e-05, 6.35415993e-05],
       [3.64559059e-05, 4.87539520e-05, 4.18273659e-05, ...,
        4.89408951e-05, 6.06918449e-05, 5.85218697e-05]],
      shape=(74, 747750))

Reading .set vs .nwb files directly from S3

Note: nwb example is not working as there's issue with the nwb file.

In [6]:
filesystem = s3fs.S3FileSystem(anon=True, client_kwargs={'region_name': 'us-east-2'})

In [7]:
def load_eeg_data_from_bids_file(bids_file_path, data_format):
    if data_format == 'set':
        eeg_data = mne.io.read_raw_eeglab(bids_file_path, verbose=False)
    elif data_format == 'nwb':
        with pynwb.NWBHDF5IO(bids_file_path, "r") as io:
            eeg_data = io.read()
    return eeg_data

def load_eeg_data_from_s3(s3path, data_format):
    with tempfile.NamedTemporaryFile(delete=False, suffix='.set') as tmp:
        with filesystem.open(s3path) as s3_file:
            tmp.write(s3_file.read())
        tmp_path = tmp.name
        eeg_data = load_eeg_data_from_bids_file(tmp_path, data_format)
        os.unlink(tmp_path)
        return eeg_data

In [8]:
%%time
load_eeg_data_from_s3('s3://testspeedeegdash/sub-002_task-FaceRecognition_eeg.set', 'set')

/var/folders/c2/k7vlgt7j1y306mj9yk_51jvr0000gn/T/ipykernel_28412/2232965347.py:3: RuntimeWarning: Unknown types found, setting as type EEG:
ekg: ['EEG063', 'EEG064']
heog: ['EEG061']
veog: ['EEG062']
  eeg_data = mne.io.read_raw_eeglab(bids_file_path, verbose=False)
/var/folders/c2/k7vlgt7j1y306mj9yk_51jvr0000gn/T/ipykernel_28412/2232965347.py:3: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  eeg_data = mne.io.read_raw_eeglab(bids_file_path, verbose=False)


CPU times: user 3.22 s, sys: 1.33 s, total: 4.55 s
Wall time: 1min 29s


/var/folders/c2/k7vlgt7j1y306mj9yk_51jvr0000gn/T/ipykernel_28412/2232965347.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  eeg_data = mne.io.read_raw_eeglab(bids_file_path, verbose=False)


<RawEEGLAB | tmphthjpz_9.set, 74 x 747750 (2991.0 s), ~422.2 MiB, data loaded>

In [11]:
%%time
load_eeg_data_from_s3('s3://testspeedeegdash/sub-002_task-FaceRecognition_eeg.nwb', 'nwb')

/Users/dtyoung/Documents/EEGLAB/EEG-Dash-Data/.venv/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.6.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/dtyoung/Documents/EEGLAB/EEG-Dash-Data/.venv/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/dtyoung/Documents/EEGLAB/EEG-Dash-Data/.venv/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.3.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


ConstructError: (root/general/extracellular_ephys/all_electrodes GroupBuilder {'attributes': {'description': 'all electrodes', 'location': array([], dtype=object), 'namespace': 'core', 'neurodata_type': 'ElectrodeGroup', 'object_id': '2a87fe4b-0cc3-458d-988b-34b55a2826d9'}, 'groups': {}, 'datasets': {}, 'links': {'device': root/general/extracellular_ephys/all_electrodes/device LinkBuilder {'builder': root/general/devices/array GroupBuilder {'attributes': {'namespace': 'core', 'neurodata_type': 'Device', 'object_id': '5f38dc95-4ad3-4700-a70a-58f803b99c82'}, 'groups': {}, 'datasets': {}, 'links': {}}}}}, "Could not construct ElectrodeGroup object due to: ElectrodeGroup.__init__: incorrect type for 'location' (got 'ndarray', expected 'str')")